<a href="https://colab.research.google.com/github/muajnstu/Multi-Class-Classification-of-YouTube-Videos-Using-A-BERT-enhanced-Machine-Learning-approach/blob/main/AlBERT_v2_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 51.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 51.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 30.2 MB/s eta 0:00:00
  Created wheel for ktrain: filename=ktrain-0.41.4-py3-none-any.whl size=25316932 sha256=791f2d755776beff4ebace5414e3a5b96a824fb8c1bce3efa86cb174eade521c
  Stored in directory: /root/.cache/pip/wheels/a6/b9/88/364e4ea6461e9f1feee834bfc2f15b92edc62600e08e3d3f5f
  C

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import string

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, AlbertTokenizer, TFAlbertModel, TFAlbertForSequenceClassification

# Load the dataset
df = pd.read_csv('https://raw.githubusercontent.com/muajnstu/ML-Datasets/refs/heads/main/Youtube%20Video%20Dataset.csv')

#category mapping
category_map = {
    "travel blog": 0,
    "science&technology": 1,
    "food": 2,
    "art&music": 3,
    "manufacturing": 4,
    "history": 5
}
df["Category"] = df["Category"].str.lower().str.strip().map(category_map)

# Shuffle the dataset
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Combine Title and Description into one text field
df['processed_text'] = df['Title'].astype(str) + " " + df['Description'].astype(str)

# Extract meaningful part using regex
def extract_txt(text):
    match = re.search(r"(?<=\s\-\s).*", str(text))
    return match.group(0) if match else text

df['processed_text'] = df['processed_text'].apply(extract_txt)

# Basic text cleaning
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\d', '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['processed_text'] = df['processed_text'].apply(clean_text)

# Rename for simplicity
df.rename(columns={'processed_text': 'text'}, inplace=True)

# Split the data
X_train, X_temp, y_train, y_temp = train_test_split(
    df['text'], df['Category'], test_size=0.2, random_state=42)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42)

# Load tokenizer for ALBERT
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

# Batch tokenization
def encode_texts(tokenizer, texts, max_length=256):
    encodings = tokenizer(
        list(texts),
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='tf'
    )
    return encodings['input_ids'], encodings['attention_mask']

# Tokenize each split
train_input_ids, train_attention_masks = encode_texts(tokenizer, X_train)
val_input_ids, val_attention_masks = encode_texts(tokenizer, X_val)
test_input_ids, test_attention_masks = encode_texts(tokenizer, X_test)

# Convert labels to tensors
y_train = tf.convert_to_tensor(y_train.values, dtype=tf.int32)
y_val = tf.convert_to_tensor(y_val.values, dtype=tf.int32)
y_test = tf.convert_to_tensor(y_test.values, dtype=tf.int32)

#Model Initialization
model = TFAlbertForSequenceClassification.from_pretrained('albert-base-v2', num_labels=6)

# Model compilation
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

# Early stopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Model training with validation data
history = model.fit(
    [train_input_ids, train_attention_masks],
    y_train,
    batch_size=16,
    validation_data=([val_input_ids, val_attention_masks], y_val),
    epochs=5,
    callbacks=[early_stopping]
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate([test_input_ids, test_attention_masks], y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Advanced evaluation: classification report and confusion matrix
y_pred_logits = model.predict([test_input_ids, test_attention_masks]).logits
y_pred = np.argmax(y_pred_logits, axis=1)

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=list(category_map.keys())))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Optionally: Save the trained model and tokenizer
model.save_pretrained('bert_youtube_classifier')
tokenizer.save_pretrained('bert_youtube_classifier')

All PyTorch model weights were used when initializing TFAlbertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFAlbertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
449/449 [==============================] - 459s 983ms/step - loss: 1.3367 - accuracy: 0.4324 - val_loss: 0.9010 - val_accuracy: 0.6601
Epoch 2/5
449/449 [==============================] - 460s 1s/step - loss: 0.8627 - accuracy: 0.6631 - val_loss: 0.8536 - val_accuracy: 0.6512
Epoch 3/5
449/449 [==============================] - 440s 980ms/step - loss: 0.8289 - accuracy: 0.6695 - val_loss: 0.8461 - val_accuracy: 0.6708
Epoch 4/5
449/449 [==============================] - 439s 979ms/step - loss: 0.8197 - accuracy: 0.6736 - val_loss: 0.8358 - val_accuracy: 0.6726
Epoch 5/5
36/36 [==============================] - 20s 564ms/step - loss: 0.8488 - accuracy: 0.6783
Test Loss: 0.8487926125526428
Test Accuracy: 0.6782531142234802
